In [7]:
from heapq import heappush, heappop, heapify

In [160]:
tmptp = 1095

In [161]:
#single link

import numpy as np

#print (up/(np.linalg.norm(a)*np.linalg.norm(b)))

def Sim_cne(N1, N2, s1, s2):  
    sums1s2 = 0
    a  = {}
    for i in s1:
        if i[0] not in a:
            a[i[0]] = i[1]
        
    for i in s2:
        if i[0] not in a:
            a[i[0]] = i[1]
        else:
            a[i[0]] += i[1]
            
    for value in a.values():
        sums1s2 += value**2
                         
    return (((1)/((N1 + N2)*(N1 + N2 -1)))*((sums1s2) -(N1+N2) ))

In [162]:
#single link

import numpy as np

#print (up/(np.linalg.norm(a)*np.linalg.norm(b)))
def Sim(cos_vec1, cos_vec2):
    
    
    vc_norm_x = [i[1] for i in cos_vec1]
    vc_norm_y = [i[1] for i in cos_vec2]

    up = 0
    
    index1 = 0
    index2 = 0
    while True:
#         print(index1, index2, cos_vec1[index1][0], cos_vec2[index2][0])
        if type(cos_vec1[index1][0])!='int':
            num1 = int(cos_vec1[index1][0])
            num2 = int(cos_vec2[index2][0])
        else:
            num1 = cos_vec1[index1][0]
            num2 = cos_vec2[index2][0]           
        
        if num1 == num2:
            up+=cos_vec1[index1][1]*cos_vec2[index2][1]
            index1+=1
            index2+=1
        else:
            if num1 >= num2:
                index2+=1
            else:
                index1+=1               
        if index1 == len(cos_vec1)-1 and index2 == len(cos_vec2)-1:
            break
        if index1 == len(cos_vec1)-1 and int(cos_vec2[index2][0]) > int(cos_vec1[index1 -1][0]):
            break
        if index2 == len(cos_vec2)-1 and int(cos_vec1[index1][0]) > int(cos_vec2[index2 -1][0]):
            break              
    down = np.linalg.norm(vc_norm_x) * np.linalg.norm(vc_norm_y)
    if down ==0:
        return 0
    else:
        return up/down

#Sim_sigle(5,7)

In [163]:
#print([(i+1) for i in range(1095)])
doc = [(i+1) for i in range(tmptp)]

doclen = len(doc)

C = []
I = [1 for i in range(doclen)] #I[n] <-1
P = []
N = [1 for i in range(tmptp)]
doctf = []

for n in doc:
    path = 'tfidf/'
    doct = [str(n)+'.txt']
    for filename in doct:
        with open(path+filename) as f:
            content = f.readlines()
            # you may also want to remove whitespace characters like `\n` at the end of each line
            content = [x.strip() for x in content] 
        content.pop(0)
        doctf.append([tuple([i.split()[0], float(i.split()[1])]) for i in content])

for n in range(doclen):
    inC = []
    inP = [] #P[n] <- priority queue for C[n] sorted on sim
        
    for i in range(doclen):
        if n ==i:
            inC.append((-1, i))
            continue #P[n].Delete(C[n][n])
#         print(n,i)
        item = (-1* round(Sim(doctf[n], doctf[i]),6), i)
        inC.append(item)

        #C[n][i] = Sim_sigle(doc[n], doc[i])
        heappush(inP, item)
        
    C.append(inC)
    P.append(inP)
    
A = []
for k in range(doclen-1):
    
    argmax = []
    
    k1 = 0
    candi = []
    for idx in range(doclen):       
        if I[idx] != 0:
            candi.append(P[idx][0][0])
        else:
            candi.append(99)
       
    k1 = candi.index(min(candi))
    
    k2 = P[k1][0][1]
    I[k2] = 0
    A.append((k1, k2))
    P[k1] = []
    
    N1N2 = (N[k1] + N[k2])
    newone = []
    index1 = 0
    index2 = 0
    while True:
        up = 0
        if type(doctf[k1][index1][0])!='int':
            num1 = int(doctf[k1][index1][0])
            num2 = int(doctf[k2][index2][0])
        else:
            num1 = doctf[k1][index1][0]
            num2 = doctf[k2][index2][0]              
        if num1 == num2:
            up+= N[k1] * doctf[k1][index1][1] +  N[k2] * doctf[k2][index2][1]
            index1+=1
            index2+=1
            newone.append((num1, up/N1N2))
        else:
            if num1 >= num2:
                newone.append((num2,  N[k2] * doctf[k2][index2][1]/N1N2))
                index2+=1
            else:
                newone.append((num1,  N[k1] * doctf[k1][index1][1]/N1N2))
                index1+=1      
        dk11 = len(doctf[k1])-1
        dk12 = len(doctf[k2])-1        
        if index1 == dk11 and dk12:
            break
        if index1 == dk11 and doctf[k2][index2][0] > doctf[k1][index1 -1][0]:
            break
        if index2 == dk12 and doctf[k1][index1][0] > doctf[k2][index2 -1][0]:
            break
    doctf[k1] = newone
    N[k1] = N1N2
    
    for i in range(doclen):
        if I[i]!= 1 or i ==k1:
            continue
        #P[i].delete(C[i][k1])
        
        P[i].remove(C[i][k1])
        heapify(P[i])
        P[i].remove(C[i][k2])
        heapify(P[i])
        
        Cik = Sim(doctf[i],  doctf[k1])
                
        #print("i, k1: ",i,k1)
        C[i][k1] = (Cik, C[i][k1][1])
        C[k1][i] = (Cik, C[k1][i][1])
          
        heappush(P[i], C[i][k1])
        heappush(P[k1], (Cik, i))
#         print("P,P: ",P[i])
#         print(P[k1])
#     print()
#     print(k)
print(A)
new_days = open('A.csv','w')
for a in A:
    for j in a:
        new_days.write(str(j) +",")
    new_days.write('\n')
new_days.close()

k1, k2:  526 528
k1, k2:  620 621
k1, k2:  661 662
k1, k2:  942 943
k1, k2:  791 795
k1, k2:  194 228
k1, k2:  475 476
k1, k2:  7 8
k1, k2:  47 48
k1, k2:  925 927
k1, k2:  190 191
k1, k2:  731 732
k1, k2:  704 705
k1, k2:  210 211
k1, k2:  563 564
k1, k2:  594 595
k1, k2:  847 848
k1, k2:  831 837
k1, k2:  154 157
k1, k2:  506 507
k1, k2:  499 500
k1, k2:  821 823
k1, k2:  100 105
k1, k2:  1033 1035
k1, k2:  242 243
k1, k2:  854 855
k1, k2:  680 683
k1, k2:  675 676
k1, k2:  196 201
k1, k2:  329 332
k1, k2:  838 839
k1, k2:  197 199
k1, k2:  582 583
k1, k2:  1084 1087
k1, k2:  304 308
k1, k2:  604 606
k1, k2:  371 380
k1, k2:  99 101
k1, k2:  824 827
k1, k2:  53 54
k1, k2:  967 968
k1, k2:  910 911
k1, k2:  895 896
k1, k2:  840 841
k1, k2:  898 905
k1, k2:  490 492
k1, k2:  230 234
k1, k2:  884 885
k1, k2:  735 736
k1, k2:  545 546
k1, k2:  953 954
k1, k2:  999 1007
k1, k2:  519 522
k1, k2:  718 721
k1, k2:  87 88
k1, k2:  995 996
k1, k2:  55 56
k1, k2:  969 970
k1, k2:  108 109
k1, k

k1, k2:  902 1031
k1, k2:  28 463
k1, k2:  868 950
k1, k2:  74 120
k1, k2:  37 565
k1, k2:  845 1058
k1, k2:  449 733
k1, k2:  503 815
k1, k2:  433 881
k1, k2:  176 220
k1, k2:  749 784
k1, k2:  115 121
k1, k2:  790 951
k1, k2:  125 687
k1, k2:  141 165
k1, k2:  389 398
k1, k2:  858 1077
k1, k2:  309 429
k1, k2:  877 880
k1, k2:  613 1075
k1, k2:  495 1013
k1, k2:  64 630
k1, k2:  471 714
k1, k2:  207 373
k1, k2:  140 617
k1, k2:  177 297
k1, k2:  148 255
k1, k2:  696 935
k1, k2:  175 739
k1, k2:  369 440
k1, k2:  464 809
k1, k2:  182 711
k1, k2:  750 780
k1, k2:  231 258
k1, k2:  273 311
k1, k2:  713 727
k1, k2:  857 1052
k1, k2:  707 932
k1, k2:  103 822
k1, k2:  425 549
k1, k2:  647 890
k1, k2:  66 86
k1, k2:  547 717
k1, k2:  1014 1015
k1, k2:  414 715
k1, k2:  1 208
k1, k2:  556 569
k1, k2:  335 772
k1, k2:  122 1021
k1, k2:  181 900
k1, k2:  640 663
k1, k2:  160 1091
k1, k2:  513 584
k1, k2:  145 313
k1, k2:  11 40
k1, k2:  355 1048
k1, k2:  384 962
k1, k2:  10 238
k1, k2:  390 1

k1, k2:  0 2
k1, k2:  5 9
k1, k2:  1 11
k1, k2:  0 12
k1, k2:  0 18
k1, k2:  0 5
k1, k2:  0 21
k1, k2:  0 6
k1, k2:  0 1
[(526, 528), (620, 621), (661, 662), (942, 943), (791, 795), (194, 228), (475, 476), (7, 8), (47, 48), (925, 927), (190, 191), (731, 732), (704, 705), (210, 211), (563, 564), (594, 595), (847, 848), (831, 837), (154, 157), (506, 507), (499, 500), (821, 823), (100, 105), (1033, 1035), (242, 243), (854, 855), (680, 683), (675, 676), (196, 201), (329, 332), (838, 839), (197, 199), (582, 583), (1084, 1087), (304, 308), (604, 606), (371, 380), (99, 101), (824, 827), (53, 54), (967, 968), (910, 911), (895, 896), (840, 841), (898, 905), (490, 492), (230, 234), (884, 885), (735, 736), (545, 546), (953, 954), (999, 1007), (519, 522), (718, 721), (87, 88), (995, 996), (55, 56), (969, 970), (108, 109), (326, 327), (530, 531), (93, 97), (129, 130), (33, 34), (501, 502), (1040, 1041), (244, 249), (921, 923), (301, 312), (454, 455), (280, 281), (974, 981), (764, 767), (614, 615), 

In [159]:
new_days = open('A.csv','w')
for a in A:
    for j in a:
        new_days.write(str(j) +",")
    new_days.write('\n')
new_days.close()
# print(i, k1, doctf[k1][index1][0])
# Sim(doctf[i],  doctf[k1])

In [149]:
if 999 >1129:
    print(123)

In [88]:
type(1)

int